In [1]:
from transformers import pipeline

/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
generator = pipeline(
    "text-generation", model="abeja/gpt2-large-japanese"
)
outputs = generator("日本で1番高い山は")
print(outputs)

/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/transformers/tokenizatio

[{'generated_text': '日本で1番高い山はご存知 「富士山」 高さが183メートルです 富士山は、その昔、天慶年間(938)藤原秀郷が富士山から献上されてから 天皇自らが登ったことから'}]


In [5]:
import pandas as pd

fill_mask = pipeline(
    "fill-mask", model="cl-tohoku/bert-base-japanese-v3"
)
masked_text = "日本の首都は[MASK]である"
outputs = fill_mask(masked_text)
display(pd.DataFrame(outputs[:3]))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


,score,token,token_str,sequence
0,0.884170,12569,東京,日本 の 首都 は 東京 で ある
1,0.024820,12759,大阪,日本 の 首都 は 大阪 で ある
2,0.020864,13017,京都,日本 の 首都 は 京都 で ある


In [7]:
masked_text = "今日の映画は刺激的で面白かった。この映画は[MASK]。"
outputs = fill_mask(masked_text)
display(pd.DataFrame(outputs[:3]))

,score,token,token_str,sequence
0,0.683934,23845,素晴らしい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 素晴らしい 。
1,0.101234,24683,面白い,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 面白い 。
2,0.048003,26840,楽しい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 楽しい 。


In [8]:
t2t_generator = pipeline(
    "text2text-generation", model="retrieva-jp/t5-large-long"
)
masked_text = "江戸幕府を開いたのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs)

/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': '徳川家康'}]


In [9]:
masked_text = "日本で通貨を発行しているのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs[0]["generated_text"])

日本銀行


In [10]:
"日本銀行" in t2t_generator.tokenizer.vocab

False

In [12]:
len(t2t_generator.tokenizer.vocab)

32100

In [13]:
# 単語とその頻度
word_freqs = {
    "たのしい": 6,
    "たのしさ": 2,
    "うつくしい": 4,
    "うつくしさ": 1,
}
# 語彙を文字で初期化
vocab = sorted(set([char for word in word_freqs for char in word]))
# 単語とその分割状態
splits = {word: [char for char in word] for word in word_freqs}
     

In [15]:
splits,  vocab

({'たのしい': ['た', 'の', 'し', 'い'],
  'たのしさ': ['た', 'の', 'し', 'さ'],
  'うつくしい': ['う', 'つ', 'く', 'し', 'い'],
  'うつくしさ': ['う', 'つ', 'く', 'し', 'さ']},
 ['い', 'う', 'く', 'さ', 'し', 'た', 'つ', 'の'])

In [16]:
from collections import Counter

def compute_most_frequent_pair(
    splits: dict[str, list[str]]
) -> tuple[str, str]:
    """
    最も頻度の高い隣接するサブワードの組を計算する
    """
    pair_freqs = Counter()  # サブワードの組のカウンタ
    for word, freq in word_freqs.items():  # すべての単語を処理
        split = splits[word]  # 現在の単語の分割状態を取得
        # すべての隣接したサブワードの組を処理
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            # サブワードの組の頻度に単語の頻度を加算
            pair_freqs[pair] += freq
    # カウンタから最も頻度の高いサブワードの組を取得
    pair, _ = pair_freqs.most_common(1)[0]
    return pair

def merge_pair(
    target_pair: tuple[str, str], splits: dict[str, list[str]]
) -> dict[str, list[str]]:
    """
    サブワードの組を結合する
    """
    l_str, r_str = target_pair
    for word in word_freqs:  # すべての単語を処理
        split = splits[word]  # 現在の単語の分割状態を取得
        i = 0
        # すべての隣接したサブワードの組を処理
        while i < len(split) - 1:
            # サブワードの組が結合対象と一致したら結合
            if split[i] == l_str and split[i + 1] == r_str:
                split = split[:i] + [l_str + r_str] + split[i + 2 :]
            i += 1
        splits[word] = split  # 現在の結合状態を更新
    return splits
     

In [17]:
for step in range(9):
    # 最も頻度の高い隣接するサブワードの組を計算
    target_pair = compute_most_frequent_pair(splits)
    # サブワードの組を結合
    splits = merge_pair(target_pair, splits)
    # 語彙にサブワードの組を追加
    vocab.append(target_pair)
     

In [18]:
print(vocab)


['い', 'う', 'く', 'さ', 'し', 'た', 'つ', 'の', ('し', 'い'), ('た', 'の'), ('たの', 'しい'), ('う', 'つ'), ('うつ', 'く'), ('うつく', 'しい'), ('し', 'さ'), ('たの', 'しさ'), ('うつく', 'しさ')]


In [21]:
from transformers import AutoTokenizer

mbert_tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-multilingual-cased"
)

print(mbert_tokenizer.tokenize("自然言語処理"))

['自', '然', '言', '語', '処', '理']


/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
print(mbert_tokenizer.tokenize("自然言語処理にディープラーニングを使う"))


['自', '然', '言', '語', '処', '理', 'に', '##ディ', '##ープ', '##ラー', '##ニング', '##を', '使', 'う']


In [23]:

xlmr_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
print(xlmr_tokenizer.tokenize("自然言語処理にディープラーニングを使う"))

['▁', '自然', '言語', '処理', 'に', 'ディー', 'プラ', 'ー', 'ニング', 'を使う']


In [24]:
bert_ja_tokenizer = AutoTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-v3"
)
print(
    bert_ja_tokenizer.tokenize("自然言語処理にディープラーニングを使う")
)
     

['自然', '言語', '処理', 'に', 'ディープ', 'ラー', '##ニング', 'を', '使う']


/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
